In [1]:
import numpy as np
from enum import Enum
from copy import deepcopy
from pathlib import Path
from tqdm import tqdm
import os
import locale
locale.setlocale(locale.LC_ALL, '')

'French_France.1252'

# Helper Function
## From viewer.py

In [2]:
HOUSE_B = 250
HOUSE_U = (750, 3000, 10000)
HOUSING = (300, 500, 650, 750)
SHOP_B = 300
SHOP_U = (2500, 10000, 50000)
INCOME = (7, 8, 9, 10)
ATTRACTION_B = 10000
ATTRACTION_U = (5000, 10000, 45000)
ATTRACTIVITY = (500, 1000, 1300, 1500)

In [3]:
class Biome(Enum):
    SEA = 0
    MOUNTAIN = 1
    PLAIN = 2

class Building(Enum):
    NONE = 0
    ATTRACTION = 1
    HOUSE = 2
    SHOP = 3
class Tile:
    def __init__(self, biome):
        self.biome = biome
        self.building = Building.NONE
        self.lvl = 0

In [4]:
def parser(path):
    f = open(path, "r")
    (rounds, money, h, w) = map(int, f.readline().split())
    carte = [ [None] * w for i in range(h) ]
    for i in range(h):
        line = f.readline()
        for j in range(w):
            carte[i][j] = Tile(Biome(int(line[j])))
    actions = []
    for i in range(rounds):
        line = f.readline()
        cur = []
        for act in line.split('|')[:-1]:
            cur.append(act.split())
        actions.append(cur)
    f.close()
    return (rounds, money, carte, actions)

In [5]:
def build(i, j, t, carte, money):
    if (i < 0 or i >= len(carte) or j < 0 or j >= len(carte[0]) \
        or carte[i][j].biome != Biome.PLAIN or carte[i][j].building != Building.NONE):
        raise Exception("build failed")
    carte[i][j].building = t
    if t == Building.ATTRACTION:
        money -= ATTRACTION_B
    elif t == Building.HOUSE:
        money -= HOUSE_B
    else:
        money -= SHOP_B
    if money < 0:
        print(i, j, t, money)
        raise Exception("build failed")
    return money

def upgrade(i, j, carte, money):
    if (i < 0 or i >= len(carte) or j < 0 or j >= len(carte[0]) or carte[i][j].biome != Biome.PLAIN \
            or carte[i][j].building == Building.NONE or carte[i][j].lvl == 3):
        raise Exception("upgrade failed")
    lvl = carte[i][j].lvl
    t = carte[i][j].building
    if t == Building.ATTRACTION:
        money -= ATTRACTION_U[lvl]
    elif t == Building.HOUSE:
        money -= HOUSE_U[lvl]
    else:
        money -= SHOP_U[lvl]
    if money < 0:
        raise Exception("upgrade failed")
    carte[i][j].lvl += 1
    return money

def destroy(i, j, carte):
    if (i < 0 or i >= len(carte) or j < 0 or j >= len(carte[0]) \
        or carte[i][j].biome != Biome.PLAIN or carte[i][j].building == Building.NONE):
        raise Exception("destroy failed")
    carte[i][j].lvl = 0
    carte[i][j].building = Building.NONE

def housing(carte):
    h = 0
    for l in carte:
        for b in l:
            if b.building == Building.HOUSE:
                h += HOUSING[b.lvl]
    return h

def attractivity(carte):
    a = 0
    for l in carte:
        for b in l:
            if b.building == Building.ATTRACTION:
                a += ATTRACTIVITY[b.lvl]
    return a

def income(pop, carte):
    i = 0
    for l in carte:
        for b in l:
            if b.building == Building.SHOP:
                i += pop * INCOME[b.lvl] // 100
    return i

def update(carte):
    return income(min(attractivity(carte), housing(carte)), carte)

In [6]:
#Unused
def simulator(path):
    (rounds, money, carte, actions) = parser(path)
    score = 0
    states = [(money, 0)]
    for i in range(len(actions)):
        carte = deepcopy(carte)
        for action in actions[i]:
            if action[0] == 'B':
                money = build(int(action[2]), int(action[3]), Building(int(action[1])), carte, money)
            elif action[0] == 'U':
                money = upgrade(int(action[1]), int(action[2]), carte, money)
            elif action[0] == 'D':
                destroy(int(action[1]), int(action[2]), carte)
        income = update(carte)
        score += income
        money += income
        states.append((money, score))
    return states

## Other

In [7]:
def count_buildable_places(carte):  # Sum up all buildable places
    count = 0
    for col in carte:
        for tile in col:
            if tile.biome == Biome.PLAIN and tile.building == Building.NONE:
                count += 1
    return count

from time import gmtime, strftime


def save_weight(score, mat):
    filename = '{}-{}.ia'.format(score, strftime("%Y-%m-%d-%H-%M-%S", gmtime()))
    with open(filename, 'wb+') as f:
        header = "{} {}\n".format(mat.shape[0], mat.shape[1])
        f.write(header.encode('utf-8'))
        np.savetxt(f, mat, fmt='%.8f')

def load_weights(directory):
    samplePop = []
    for file in os.listdir(directory):
        if file != '.DS_Store':
            f = open(directory+'/'+file, 'rb')
            samplePop.append(np.loadtxt(f))

    np.save('gen3-{}.npy'.format(29), np.array(samplePop))

# Building Neural Net (1 layer)


In [8]:
# GLOBAL VARIABLE

buildings = [Building.HOUSE, Building.SHOP, Building.ATTRACTION]
prices = [HOUSE_B, SHOP_B, ATTRACTION_B]
prices_up = [HOUSE_U, SHOP_U, ATTRACTION_U]

is_full = False
lvl_upgrade = [0,0,0]

NUM_OUTPUT = 6
NUM_INPUT = 5

initial_money = 0
max_buildable_tile = 0




In [9]:
def random_init(inp, out, mu, sigma):  # Init random weights for the population
    return (np.random.normal(mu, sigma, size=(inp, out))) # Tuning can be done

def format_input(money, income, carte, builts): # Input for the neural net  [see C# source code MyBot.cs]
    s = np.sum(builts)
    
    global lvl_upgrade
    global is_full
    if s == max_buildable_tile:
        is_full = True
    else:
        lvl_upgrade =  [0]*3
    if s == 0:
        return np.array([money/initial_money, income/initial_money, 1/3,1/3,1/3])
    return np.array([money/initial_money, income/initial_money, builts[0]/s, builts[1]/s, builts[2]/s])

def try_build(t, nb, money, carte, builts):# built anywhere possible (since position doesnt matter) [MyBot.cs]
    n = 0
    k = len(carte)
    l = len(carte[0])
    while money > prices[t] and n < nb:    
        for i in range(0, k):
            built = False
            for j in range(0, l):
                if carte[i][j].biome == Biome.PLAIN and carte[i][j].building == Building.NONE:
                    carte[i][j].building = buildings[t]
                    money -= prices[t]
                    #print ("B ", t, i, j, money)
                    builts[t] += 1
                    
                    built = True
                    break
            if built: break
        
        n += 1
    return money


def try_upgrade(t, nb, money, carte): # upgrade building [MyBot.cs]
    n = 0
    k = len(carte)
    l = len(carte[0])
    #lvl = 0 # first upgrade every building from lvl 0 -> 1
    if lvl_upgrade[t] > 2:  # Avoid looping through the entire map 3 times for nothing
        return False
    while money > prices_up[t][lvl_upgrade[t]] and n < nb: 
        for i in range(0, k):
            upgraded = False
            for j in range(0, l):
                if carte[i][j].building == buildings[t] and carte[i][j].lvl == lvl_upgrade[t]:
                    carte[i][j].lvl += 1
                    money -= prices_up[t][lvl_upgrade[t]]
                    #print ("U ", t,lvl, i, j, money)

                    upgraded = True
                    break
            if upgraded: break
        if upgraded: 
            n += 1
        elif lvl_upgrade[t] < 2:
            #lvl += 1
            lvl_upgrade[t] += 1
        else: 
            break
    return money

def activation_func(output, money, carte, builts): # Interprete the output of the neural net  [MyBot.cs]
    for i in range(0, NUM_OUTPUT//2): # Build
        y = int(output[i])
        if y > 0 and money > prices[i] and not is_full:  # ~ rect linear
            #print(y, i)
            money = try_build(i, y, money, carte, builts)
    for i in range(NUM_OUTPUT//2, NUM_OUTPUT): # Upgrade
        y = int(output[i])
        if y > 0 and money > prices_up[i%3][0]:
            #print(y, i)
            money = try_upgrade(i%3, y, money, carte)
    return money

def play(w):
    rounds, money, _carte, actions = parser('bot.out') # Getting initial data from bot.out
    carte = _carte[:]
    income = 0
    global initial_money
    global max_buildable_tile
    
    #Optimizing try_build and try_upgrade
    global is_full
    global lvl_upgrade
    is_full = False
    lvl_upgrade = [0,0,0] 
    
    max_buildable_tile = count_buildable_places(carte)
    
    builts = [0]*3 # Nombre de batiments construits par l'IA
    
    initial_money = money

    score = 0
    for r in range(0, rounds):
        #print (r)
        X = format_input(money, income,  carte, builts)[:]
        Y = np.dot(X, w)# + b
        Y = np.transpose(Y)
        #print (Y)
        
        money = activation_func(Y, money, carte, builts)
        income = update(carte)
        #print(r+1, money, income)
        money += income
        score += income
       
    return score

## Evolutionnary Algo

In [10]:
def board(logs, number_survive):  # Show best score each generation and select the more performing weights
    logs.sort(key=lambda tup: tup[0])
    zeros = 0
    moy = 0
    l = len(logs)
    bests = []
    print("best : {0:n}".format(logs[l-1][0]))
    for i in range(0, l):
        if logs[i][0] == 0:
            zeros += 1
        if i > (l-number_survive):
            bests.append(logs[i][1])
        moy += logs[i][0]
    print("moy :", moy/l)
    print("zero prop : {0:.0f}%".format(zeros/l*100))
    return bests

def breed(parents, nb_pop): # Create genetical (weights) children for the selected parents
    L = len(parents)
    child = []
    s = parents[0].shape
    for i in range(0, nb_pop-L):
        index = np.random.choice(L, (2))
        child.append( np.choose(np.random.choice(2, s), choices=[parents[index[0]], parents[index[1]]] ) )
    child.extend(parents)
    return child

def mutate(pop, mult, rare): # Randomly change neural net weights in the population
    L = len(pop)
    s = pop[0].shape
    for p in range(0, L):
        if np.random.randint(rare) == 0:
            i = np.random.randint(s[0])
            j = np.random.randint(s[1])
            pop[p][i][j] += (np.random.random()-0.5)*mult
    return pop
            

## First try : < 1 000 000

| Input   | Output (To Build) |
|---------|-------------------|
| Money   |    Houses         |
|Income   |    Shops          |
|Nb empty |    Attractions    |


## Second try : Normalizing data



* Using building proportion : `~9 000 000`
<br/>
$$Norm(money)=\frac{money}{initial\ money}$$
<br/>

<br/>
$$Norm(nb\ houses)=\frac{nb\ houses}{nb\ buildings}$$
<br/>

| Input   | Output (To Build) |
|---------|-------------------|
| Money   |    Houses         |
|Income   |    Shops          |
|Nb houses|    Attractions    |
|Nb shops |
|Nb Attrac|

* Adding Upgrade : `~29 000 000`

| Input   | Output            |
|---------|-------------------|
| Money   |    Houses  (Build)|
|Income   |    Shops   (Build)|
|Nb houses|Attractions (Build)|
|Nb shops |    Houses  (Up)   |
|Nb Attrac|    Shops   (Up)   |
|         |Attractions (Up)   |


In [11]:
def genetic_algo(n, nb_pop, mutate_rare, mutate_mult, keep, mu, sigma): # Training
    
    pop = []
    maX_log = []
    for gen in range(0, n):
        logs = []
        maX = 0
        maX_W = []
        pop_len = len(pop)
        
        # Building
        pop.extend([random_init(NUM_INPUT, NUM_OUTPUT, mu, sigma)[:] for i in range(0, nb_pop-pop_len)])
        
        # Mutating
        pop = mutate(pop, mutate_mult, mutate_rare)[:]
        for i in tqdm(range(0, nb_pop)):
            p = pop[i]
            score = play(p)
            logs.append((score, p[:]))
            if score > maX:
                maX = score
                maX_W = p[:]
        best = board(logs, keep)[:] # Selecting
        maX_log.append(maX)
        
        # Saving
        if gen == n-1:
            np.save('gen-{}.npy'.format(maX), best) # Saving Batch
            break
        
        # Breeding
        pop = breed(best, nb_pop)[:]
        
    return maX, maX_W, maX_log

In [12]:
s, w, logs = genetic_algo(20, 500, 6, 2.0, 10, 0.5, 2.25)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:08<00:00, 61.81it/s]


best : 154 616
moy : 1418.044
zero prop : 93%


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:08<00:00, 59.65it/s]


best : 9 312 660
moy : 140017.786
zero prop : 48%


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:12<00:00, 41.51it/s]


...

best : 27 355 525
moy : 26993189.276
zero prop : 0%


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:25<00:00, 19.33it/s]


best : 27 357 209
moy : 26877719.546
zero prop : 0%


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:25<00:00, 19.64it/s]


best : 27 357 209
moy : 26823186.124
zero prop : 1%
